<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/LSTM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 新段落

In [ ]:
cd gdrive/My Drive/Colab_Notebooks/code


/content/gdrive/My Drive/Colab_Notebooks/code


In [ ]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [2]:
pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 8.6MB/s 
     |████████████████████████████████| 133kB 28.3MB/s 
     |████████████████████████████████| 7.2MB 22.7MB/s 
     |████████████████████████████████| 71kB 12.0MB/s 
ERROR: botocore 1.19.44 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
a = "i am a dog"
a_tokens = bert_tokenizer.tokenize(a)
print(a_tokens)
a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
print(a_seq_ids)
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
out,_ = bert_model(batch_data)
print(out[0].shape)
print(out[0][0][0])

In [ ]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
import numpy as np
import csv
label_List = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
important_list = ["Low",
        "Medium",
        "High",
        "Critical"
       ]
tweets = []
path = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention/data/ll.csv'
with open(path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        tweet_full = line
        tweets.append({
            'id': tweet_full[0],
            'label':tweet_full[1],
            'important':tweet_full[2],
            'text': tweet_full[3].lower(),
            # 'name': tweet_full['user']['name'].split()[0]
            })
tweet_list = []
text_list = []
imformation_list = []
important_list = []
label_list = []
q = 0
for i in range(len(tweets)):
        #text_list.append(tweets[i]['text'])
        a_tokens = bert_tokenizer.tokenize(tweets[i]['text'])
        a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
        batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
        bert_model.eval()
        out,_ = bert_model(batch_data)
        #print(type(out[0][0]))
        print(q)
        if q == 3000:
          break
        tweets_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        tweets_label = eval(tweets[i]['label'])
        tweets_important = tweets[i]['important']
        p = 0
        for j in label_List:
            for k in tweets_label:
                if k == j:
                    tweets_vector[p] = 1
            p+=1
        for q in important_List:
            for l in tweets_label:
                if l == q:
                    tweets_vector[p] = 1
            p+=1
        label_list.append(tweets_vector)
        text_list.append(out[0][0].cpu())
        q+=1


In [ ]:
cd /content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention

/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention


In [ ]:
a = np.array(label_list)
b = np.array(text_list)

In [ ]:
np.save('./data/Bert_label1.npy',a)
np.save('./data/Bert_text1.npy',b)

In [ ]:
c = np.load('./data/Bert_text1.npy',allow_pickle=True)
d = c.tolist()
